# IRIS - Connection II

![Add layer](https://github.com/DrKAI/image/blob/main/Add%20Layer.png?raw=true)

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from sklearn.datasets import load_iris

## 데이터 불러오기
* iris.feature_names를 이용하여 판다스 데이터 프레임 만들기!

In [ ]:
iris = load_iris()

x = iris.data
y = iris.target

In [ ]:
df_x = pd.DataFrame(x, columns=iris.feature_names)

df_x.head()

## train set, test set 구분하기

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(df_x, y, test_size=0.2, random_state=2021)

In [ ]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

## **sepal끼리, petal끼리**

In [ ]:
print(df_x.columns)

In [ ]:
tr_x_se = train_x.loc[:, ['sepal length (cm)', 'sepal width (cm)'] ]
tr_x_pe = train_x.loc[:, ['petal length (cm)', 'petal width (cm)'] ]

In [ ]:
print(tr_x_se.shape)
print(tr_x_pe.shape)

In [ ]:
te_x_se = test_x.loc[:, ['sepal length (cm)', 'sepal width (cm)'] ]
te_x_pe = test_x.loc[:, ['petal length (cm)', 'petal width (cm)'] ]

## One-hot Encoding

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
train_y = to_categorical(train_y, 3)
test_y = to_categorical(test_y, 3)


In [ ]:
train_y.shape

## Modeling : multi-input & Add layer

In [ ]:
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Input, Dense, Add
from tensorflow.keras.activations import relu, softmax

from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [ ]:
# 1. 세션 클리어
clear_session()

# 2. 레이어 사슬처럼 엮기 : input 2개!
il_se = Input( shape=(2,) )
hl_se = Dense(4, activation=relu)(il_se)

il_pe = Input( shape=(2,) )
hl_pe = Dense(4, activation=relu)(il_pe)

al = Add()([hl_se, hl_pe])
ol = Dense(3, activation=softmax)(al)

# 3. 모델 시작과 끝 지정
model = Model([il_se, il_pe], ol)

# 4. 모델 컴파일
model.compile(loss=categorical_crossentropy, metrics=['accuracy'],
              optimizer=Adam())

## 모델 시각화

In [ ]:
plot_model(model, show_shapes=True)

## 학습 시켜보기!

In [ ]:
es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=5,
                   verbose=1,
                   restore_best_weights=True)

In [ ]:
model.fit([tr_x_se, tr_x_pe], train_y, validation_split=0.15,
          epochs=1000, verbose=1, callbacks=[es])